# Assignment

Assigment is in the intersection of Named Entity Recognition and Dependency Parsing.

0. Evaluate spaCy NER on CoNLL 2003 data (provided)
    - report token-level performance (per class and total)
        - accuracy of correctly recognizing all tokens that belong to named entities (i.e. tag-level accuracy) 
    - report CoNLL chunk-level performance (per class and total);
        - precision, recall, f-measure of correctly recognizing all the named entities in a chunk per class and total  

1. Grouping of Entities.
Write a function to group recognized named entities using `noun_chunks` method of [spaCy](https://spacy.io/usage/linguistic-features#noun-chunks). Analyze the groups in terms of most frequent combinations (i.e. NER types that go together). 

2. One of the possible post-processing steps is to fix segmentation errors.
Write a function that extends the entity span to cover the full noun-compounds. Make use of `compound` dependency relation.

# CoNLL Data
From https://www.clips.uantwerpen.be/conll2003/ner/

The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on four types of named entities: persons, locations, organizations and names of miscellaneous entities that do not belong to the previous three groups. 

The CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on a separate line and there is an empty line after each sentence. The first item on each line is a word, the second a part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags and the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only if two phrases of the same type immediately follow each other, the first word of the second phrase will have tag B-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase.


In [1]:
import conll
import my_ents

# token, POS tag, syntactic chunk tag (IOB), entity tag (IOB)
test = conll.read_corpus_conll('dataset/train.txt')
test = [my_ents.Sentence(sent) for sent in test if '-DOCSTART-' not in sent[0][0]]
# test = test[:100]

# count_ = [len(sent) for sent in test]
# print(sum(count_))

# 0. Evaluate spaCy NER on CoNLL 2003 data (provided)
- report token-level performance (per class and total)
    - accuracy of correctly recognizing all tokens that belong to named entities (i.e. tag-level accuracy) 
- report CoNLL chunk-level performance (per class and total);
    - precision, recall, f-measure of correctly recognizing all the named entities in a chunk per class and total

Conversion from Ontonotes tags to CoNLL format

In [2]:
# conversion of tags from Ontonotes (spacy) to CoNLL format
def from_spacy_to_conll(predictions_spacy):
    switcher = {
                ' ': '',
                '': '',
                'ORG': '-ORG',
                'PER': '-PER',
                'LOC': '-LOC',
                'PERSON': '-PER',
                'GPE': '-LOC'
            }
    
    # LOC, PER, ORG, MISC
    predictions = []
    
    for sent in predictions_spacy:
        new = []
        
        for ent in sent:
            # merge iob and entity type
            new.append((ent.text, ent.ent_iob_ + switcher.get(ent.ent_type_, '-MISC')))
        
        predictions.append(new)
        
    return predictions

## Custom tokenizer
Define custom tokenizer for spacy, otherwise spaCy will tokenize differently from how the CoNLL dataset has been tokenized. This would produce different tokens in output, rendering impossible to compute the accuracy.

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

# def tokenizer_(sent):
#     return spacy.tokens.Doc(nlp.vocab, sent.split())

nlp.tokenizer = lambda sent: spacy.tokens.Doc(nlp.vocab, sent.split())

In [4]:
%%time

# spacy predictions
predictions = [nlp(str(sent)) for sent in test]

CPU times: user 1min 23s, sys: 103 ms, total: 1min 23s
Wall time: 1min 23s


In [5]:
# convert to NLTK format so that conll.evaluate can be used
predictions = from_spacy_to_conll(predictions)

predictions[6]

[('He', 'O'),
 ('said', 'O'),
 ('further', 'O'),
 ('scientific', 'O'),
 ('study', 'O'),
 ('was', 'O'),
 ('required', 'O'),
 ('and', 'O'),
 ('if', 'O'),
 ('it', 'O'),
 ('was', 'O'),
 ('found', 'O'),
 ('that', 'O'),
 ('action', 'O'),
 ('was', 'O'),
 ('needed', 'O'),
 ('it', 'O'),
 ('should', 'O'),
 ('be', 'O'),
 ('taken', 'O'),
 ('by', 'O'),
 ('the', 'B-ORG'),
 ('European', 'I-ORG'),
 ('Union', 'I-ORG'),
 ('.', 'O')]

## Token-level accuracy, total and per-class

In [6]:
# organize test data in tuples (entity, tag)
test_set = [[(ent.text, ent.ent_tag) for ent in sent.ents] for sent in test]

test_set[6]

[('He', 'O'),
 ('said', 'O'),
 ('further', 'O'),
 ('scientific', 'O'),
 ('study', 'O'),
 ('was', 'O'),
 ('required', 'O'),
 ('and', 'O'),
 ('if', 'O'),
 ('it', 'O'),
 ('was', 'O'),
 ('found', 'O'),
 ('that', 'O'),
 ('action', 'O'),
 ('was', 'O'),
 ('needed', 'O'),
 ('it', 'O'),
 ('should', 'O'),
 ('be', 'O'),
 ('taken', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('European', 'B-ORG'),
 ('Union', 'I-ORG'),
 ('.', 'O')]

### Total accuracy

In [7]:
def total_accuracy(predictions_labels, labels):
    if len(predictions_labels) != len(labels):
        raise Exception('Prediction labels and test labels have different lenght')
    
    correct = 0
    for i in range(len(predictions_labels)):
        if predictions_labels[i] == labels[i]:
            correct += 1
    
    return correct/len(labels)

pred_labels = [ent[1] for sent in predictions for ent in sent]
test_labels = [ent[1] for sent in test_set for ent in sent]

total_accuracy(pred_labels, test_labels)

0.8257154222796274

### Per-class accuracy

In [13]:
from sklearn.metrics import classification_report
import pandas as pd

per_class_metrics = classification_report(test_labels, pred_labels, output_dict=True)
pd_tbl_class = pd.DataFrame().from_dict(per_class_metrics).transpose()
pd_tbl_class

,precision,recall,f1-score,support
B-LOC,0.804400,0.706723,0.752404,7140.000000
B-MISC,0.127121,0.573008,0.208080,3438.000000
B-ORG,0.463032,0.313083,0.373572,6321.000000
B-PER,0.794085,0.650909,0.715404,6600.000000
I-LOC,0.631902,0.623163,0.627502,1157.000000
I-MISC,0.042681,0.251948,0.072996,1155.000000
I-ORG,0.484502,0.548596,0.514561,3704.000000
I-PER,0.815482,0.800353,0.807847,4528.000000
O,0.952459,0.873781,0.911425,169578.000000
accuracy,0.825715,0.825715,0.825715,0.825715


### Chunk-level accuracy

In [9]:
results = conll.evaluate(test_set, predictions)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
MISC,0.120,0.541,0.196,3438
PER,0.771,0.632,0.695,6600
LOC,0.796,0.699,0.744,7140
ORG,0.413,0.279,0.333,6321
total,0.406,0.544,0.465,23499
